# Notebook: 5 KCs toy example

## Package import

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac
sys.path.append("/home/olivier/PycharmProjects/bayesian-kst/")  # for ubuntu

from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.resources_layer.exercise_family import ExerciseFamily
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.evaluation import Evaluation
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
from sklearn.metrics import roc_auc_score


## Domain knowledge model
We define a set of 5 KCs (A, B, C, D and E) and the relationships that rule them ($A \longrightarrow C$, $B \longrightarrow C$, $C \longrightarrow D$, $C \longrightarrow E$). We suppose that the prerequisites $A \longrightarrow C$ and $C \longrightarrow D$ are strong while $B \longrightarrow C$ and $C \longrightarrow E$ are weak. Figure \ref{fig:symbolic_model_5_kcs} represents the domain knowledge model of this 5 KCs example.

In [2]:
# we define the KCs
KC_A = KnowledgeComponent(1, "A")
KC_B = KnowledgeComponent(2, "B")
KC_C = KnowledgeComponent(3, "C")
KC_D = KnowledgeComponent(4, "D")
KC_E = KnowledgeComponent(5, "E")

A_2_C = Link(source=KC_A, target=KC_C)
B_2_C = Link(source=KC_B, target=KC_C)
C_2_D = Link(source=KC_C, target=KC_D)
C_2_E = Link(source=KC_C, target=KC_E)

domain = Domain([KC_A, KC_B, KC_C, KC_D, KC_E], [A_2_C, B_2_C, C_2_D, C_2_E])

params = {"slip": .01, "guess":.01}

# there are 5 exercises corresponding to KC A
ex_1 = Exercise(1, KC_A, "qcm", ex_content="", params=params)
ef_1 = ExerciseFamily(1, "ex fam 1", kc=KC_A, exercise_list=[ex_1])
ex_2 = Exercise(2, KC_B, "qcm", ex_content="", params=params)
ef_2 = ExerciseFamily(2, "ex fam 2", kc=KC_B, exercise_list=[ex_2])
ex_3 = Exercise(3, KC_C, "qcm", ex_content="", params=params)
ef_3 = ExerciseFamily(3, "ex fam 3", kc=KC_C, exercise_list=[ex_3])
ex_4 = Exercise(4, KC_D, "qcm", ex_content="", params=params)
ef_4 = ExerciseFamily(4, "ex fam 4", kc=KC_D, exercise_list=[ex_4])
ex_5 = Exercise(5, KC_E, "qcm", ex_content="", params=params)
ef_5 = ExerciseFamily(5, "ex fam 5", kc=KC_E, exercise_list=[ex_5])

link_strengths = {KC_A:{KC_C: 'strong'}, KC_B:{KC_C: 'weak'}, KC_C: {
    KC_A: 'strong', KC_B: 'weak', KC_D: 'strong', KC_E: 'weak'}, KC_D:{KC_C: 'strong'}, KC_E:{KC_C: 'weak'}}

priors = {'A': 0.1, 'B': 0.1, 'C': 0.1, 'D':.1, 'E':.1}

## Use case 1: learner lacking of prerequisite mastering
Let define a learner $L_0$ as following: we suppose $L_0$ greatly masters KC $A$ but because he doesn't master KC $B$, he can't correctly answer to exercises on KC $C$ and, therefore, on KC $D$ and $E$ neither.

In [9]:
learner = Learner(1, None)
learner_pool = LearnerPool(domain, link_strengths)
learner_pool.add_learner(learner)
print(priors)
knowledge_state = learner.predict_next_step(priors, evaluation=[KC_A, True], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.1, 'B': 0.1, 'C': 0.1, 'D': 0.1, 'E': 0.1}
{'(A)t': [0.1, 0.9]}
{'A': 0.8336986128968591, 'B': 0.46241361880851883, 'C': 0.5150198986265169, 'D': 0.22223177228052968, 'E': 0.20227437401007342, 'eval(A)': 0.7669588903174872, 'eval(B)': 0.4699308950468151, 'eval(C)': 0.5120159189012135, 'eval(D)': 0.2777854178244238, 'eval(E)': 0.26181949920805875}


In [10]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_B, False], pred_mode='one_kc')
print(knowledge_state)

{'(B)t': [0.9, 0.1]}
{'A': 0.8702216894238379, 'B': 0.2058170645553604, 'C': 0.47107330282888343, 'D': 0.19896133933623222, 'E': 0.19767094166614227, 'eval(A)': 0.7961773515390703, 'eval(B)': 0.26465365164428833, 'eval(C)': 0.47685864226310687, 'eval(D)': 0.25916907146898577, 'eval(E)': 0.25813675333291386}


In [11]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_C, False], pred_mode='one_kc')
print(knowledge_state)

{'(C)t': [0.9, 0.1]}
{'A': 0.8530352642476012, 'B': 0.3375260936560451, 'C': 0.1475153261602272, 'D': 0.08824669955480093, 'E': 0.1210341548035097, 'eval(A)': 0.7824282113980808, 'eval(B)': 0.3700208749248361, 'eval(C)': 0.21801226092818174, 'eval(D)': 0.17059735964384076, 'eval(E)': 0.19682732384280777}


In [87]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_C, False], pred_mode='one_kc')
print(knowledge_state)

{'(C)t': [0.9, 0.1]}
{'A': 0.6024568063160565, 'B': 0.6051521982968396, 'C': 0.008584255369837547, 'D': 0.002564212698675351, 'E': 0.00818307735413778, 'eval(A)': 0.5819654450528452, 'eval(B)': 0.5841217586374717, 'eval(C)': 0.10686740429587005, 'eval(D)': 0.1020513701589403, 'eval(E)': 0.10654646188331023}


## Use case 2: learner already mastering the most advanced KCs
Let define another learner $L_2$ as following: we suppose $L_1$ has already studied the domain knowledge and therefore greatly masters every KCs but KC $E$ he hasn't studied yet.

In [6]:
learner = Learner(2, None)
learner_pool = LearnerPool(domain, link_strengths)
learner_pool.add_learner(learner)

knowledge_state = learner.predict_next_step(priors, evaluation=[KC_C, True], pred_mode='one_kc')
print(knowledge_state)

{'(C)t': [0.1, 0.9]}
{'A': 0.7184325206081045, 'B': 0.585694177770173, 'C': 0.7411165243326077, 'D': 0.3139238686022544, 'E': 0.275030944595098, 'eval(A)': 0.6747460164864836, 'eval(B)': 0.5685553422161383, 'eval(C)': 0.6928932194660862, 'eval(D)': 0.3511390948818035, 'eval(E)': 0.3200247556760784}


In [7]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_D, False], pred_mode='one_kc')
print(knowledge_state)

{'(D)t': [0.9, 0.1]}
{'A': 0.87223067841775, 'B': 0.7735329009106301, 'C': 0.7486651522311646, 'D': 0.05660918756212582, 'E': 0.2612793973653431, 'eval(A)': 0.7977845427342001, 'eval(B)': 0.7188263207285042, 'eval(C)': 0.6989321217849316, 'eval(D)': 0.14528735004970067, 'eval(E)': 0.3090235178922745}


In [8]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_E, False], pred_mode='one_kc')
print(knowledge_state)

{'(E)t': [0.9, 0.1]}
{'A': 0.9110169401771586, 'B': 0.8484487544363444, 'C': 0.7165499916386735, 'D': 0.05377872818401954, 'E': 0.04667011398484879, 'eval(A)': 0.8288135521417269, 'eval(B)': 0.7787590035490756, 'eval(C)': 0.6732399933109388, 'eval(D)': 0.1430229825472156, 'eval(E)': 0.13733609118787904}
